In [3]:
import numpy as np
from scipy.optimize import minimize

In [22]:
base_price=[10,10,10]
target_trade_spend = 10515.58
intercept = 15
coef_1 =-3
coef_2 =0.004
target_edlp_spend = 1906.41
target_tpr_spend = 8609.172

In [32]:
def objective(x):
    
    mid = int(len(x)/3)
    total = 0
    unit_sales = list(np.exp((np.log(x[i])*coef_1)*x[i+mid+mid]
             +(np.log(base_price[i])*coef_1+x[i+mid]*coef_2)*(1-x[i+mid+mid])
             +intercept) for i in range(mid))
    
    price = list((base_price[i]*(1-x[i+mid]/100)*(1-x[i+mid+mid])) 
        +(x[i]*x[i+mid+mid]) for i in range(mid))
    
    dollar_sales = [a*b for a,b in zip(unit_sales,price)]
    
    return -sum(dollar_sales)
objective([10,10,10,10,10,10,1,1,0])

-96002.20397006179

In [35]:
def constraint1(x):
    
    mid = int(len(x)/3)
    total = 0
    unit_sales = list(np.exp((np.log(x[i])*coef_1)*x[i+mid+mid]
             +(np.log(base_price[i])*coef_1+x[i+mid]*coef_2)*(1-x[i+mid+mid])
             +intercept) for i in range(mid))
    
    price = list((base_price[i]*(1-x[i+mid]/100)*(1-x[i+mid+mid])) 
        +(x[i]*x[i+mid+mid]) for i in range(mid))
    
#     dollar_sales = [a*b for a,b in zip(unit_sales,price)]
    trade_spent = [a-b for a,b in zip(base_price, price)]

    total_trade_spent = [a*b for a,b in zip(trade_spent, unit_sales)]
    return sum(total_trade_spent) - target_trade_spend
constraint1([10,10,10,10,10,10,1,1,0])

-7113.151497708937

In [27]:
def constraint2(x):
    mid = int(len(x)/3)
    total = 0
    unit_sales = list(np.exp((np.log(x[i])*coef_1)*x[i+mid+mid]
             +(np.log(base_price[i])*coef_1+x[i+mid]*coef_2)*(1-x[i+mid+mid])
             +intercept) for i in range(mid))
    
    price = list((base_price[i]*(1-x[i+mid]/100)*(1-x[i+mid+mid])) 
        +(x[i]*x[i+mid+mid]) for i in range(mid))
    
    trade_spent = [a-b for a,b in zip(base_price, price)]

    total_trade_spent = [a*b for a,b in zip(trade_spent, unit_sales)]

    edlp_trade_spent = [a*b for a,b in zip(total_trade_spent, x[mid+mid:])]
    
    return sum(edlp_trade_spent) - target_edlp_spend
constraint2([9.5,10,10,10,10,10])

-632204.2415494585

In [28]:
def constraint3(x):
    mid = int(len(x)/3)
    total = 0
    unit_sales = list(np.exp((np.log(x[i])*coef_1)*x[i+mid+mid]
             +(np.log(base_price[i])*coef_1+x[i+mid]*coef_2)*(1-x[i+mid+mid])
             +intercept) for i in range(mid))
    
    price = list((base_price[i]*(1-x[i+mid]/100)*(1-x[i+mid+mid])) 
        +(x[i]*x[i+mid+mid]) for i in range(mid))
    
    trade_spent = [a-b for a,b in zip(base_price, price)]

    total_trade_spent = [a*b for a,b in zip(trade_spent, unit_sales)]

    edlp_trade_spent = [a*(1-b) for a,b in zip(total_trade_spent, x[mid+mid:])]
    return sum(edlp_trade_spent) - target_tpr_spend
constraint3([9.5,10,10,10,10,10])

558658.8763945126

In [29]:
n = 9
x0 = np.zeros(n)
x0[0] = 10.0
x0[1] = 10.0
x0[2] = 10.0
x0[3] = 10.0
x0[4] = 10.0
x0[5] = 10.0
x0[6] = 0.0
x0[7] = 0.0
x0[8] = 1.0

# show initial objective
print('Initial Total Sales Objective: ' + str(-objective(x0)))

Initial Total Sales Objective: 93933.88676596023


In [31]:
# optimize
b1 = (9.5,10.0)
b2 = (5,50)
b3 = (0,1)
bnds = (b1, b1, b1, b2, b2, b2,b3,b3,b3)
con1 = {'type': 'eq', 'fun': constraint1}
con2 = {'type': 'eq', 'fun': constraint2}
con3 = {'type': 'eq', 'fun': constraint3}

cons = ([con1,con2,con3])
solution = minimize(objective,x0,method='SLSQP',bounds=bnds,constraints=cons)
x = solution.x

# show final objective
print('Final Total Sales Objective: ' + str(-objective(x)))

# print solution
print('Solution')
print('edlp1 = ' + str(x[0]))
print('edlp2 = ' + str(x[1]))
print('edlp3 = ' + str(x[2]))
print('tpr1 = ' + str(x[3]))
print('tpr2 = ' + str(x[4]))
print('tpr3 = ' + str(x[5]))
print('flag1 = ' + str(x[6]))
print('flag2 = ' + str(x[7]))
print('flag3 = ' + str(x[8]))

Final Total Sales Objective: 93933.88676596023
Solution
edlp1 = 10.0
edlp2 = 10.0
edlp3 = 10.0
tpr1 = 10.0
tpr2 = 10.0
tpr3 = 10.0
flag1 = 0.0
flag2 = 0.0
flag3 = 1.0
